In [1]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy import*
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()
from Utilities.kukaKinematics import *
from Utilities.symbolicFunctions import *

In [2]:
# #Funksjoner som trengs

# def PsFromTsd(T_sd):
#     #Finner Ps fra T_sd
#     #T_sd gir konfigurasjonen vi vil ha end-effector framen, B, i.
#     #B, og derav også M, er lik som i DH
#     #s er plassert nederst på roboten med positiv z oppover, altså ikke som i DH. Bør kanskje endres til å være lik DH 
#     P_d = np.array([0,0,80,1])
#     P_s = T_sd@P_d

#     return P_s

# def exp3(omega, theta):
#     omega = skew(omega)
#     R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
#     return R

# def skew(v):
#     return Matrix([[0, -v[2], v[1]],
#                     [v[2], 0, -v[0]],
#                     [-v[1], v[0], 0]])
                    
# def exp6(twist, theta):
#     omega = skew(twist[:3])
#     v = Matrix(twist[3:])
#     T = eye(4)
#     T[:3,:3] = exp3(twist[:3], theta)
#     T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
#               (theta-sin(theta)) * omega * omega) * v
#     return T


# def rotX(alfa_im1):
#     Rx = sp.eye(4)
#     Rx[1,1] =    sp.cos(alfa_im1)
#     Rx[1,2] =   -sp.sin(alfa_im1)
#     Rx[2,1] =    sp.sin(alfa_im1)
#     Rx[2,2] =    sp.cos(alfa_im1)
#     return Rx

# def rotZ(alfa_im1):
#     Rz = sp.eye(4)
#     Rz[0,0] =    sp.cos(alfa_im1)
#     Rz[0,1] =   -sp.sin(alfa_im1)
#     Rz[1,0] =    sp.sin(alfa_im1)
#     Rz[1,1] =    sp.cos(alfa_im1)
#     return Rz

# def rotY(alfa_im1):
#     Ry = sp.eye(4)
#     Ry[0,0] =    sp.cos(alfa_im1)
#     Ry[0,2] =    sp.sin(alfa_im1)
#     Ry[2,0] =   -sp.sin(alfa_im1)
#     Ry[2,2] =    sp.cos(alfa_im1)
#     return Ry




In [93]:

def Slist_maker(omega_mat, q_mat): #omega_mat and q_mat of type matrix with q_i & omega_i as columns
    #Returns v_mat in same type/format
    v_mat = calc_v(omega_mat, q_mat)    
    n_joints = omega_mat.shape[1]
    Slist = sp.zeros(6, n_joints)
    
    for i in range(n_joints):
        Slist[:3,i] = omega_mat[:,i]
        Slist[3:,i] = v_mat[:,i]
    return Slist

def calc_v(omega_mat, q_mat):
    #omega_mat and q_mat of type matrix with q_i & omega_i as columns
    #Returns v_mat in same type/format
    assert len(omega_mat) == len(q_mat)
    
    n_joints = omega_mat.shape[1] 
    v_mat = sp.zeros(3, n_joints)      

    for i in range(n_joints):
        v_mat[:,i] = (-skew(omega_mat.col(i)) * q_mat.col(i))
    return v_mat


def apply_joint_lim(jointLimits, thetas):
    ''' Check if Inverse Kinematics solution (thetas) is within jointlimits\n 
    PARAMETERS:
    Jointlimits, numpy 2D array with lower and upper limits in deg\n
    Jointangles, numpy 1D array in rad
    RETURNS: Boolean true or false, if false a print message with the offending link is printed '''
    jointLimits = np.deg2rad(jointLimits) 
    #thetas %= 2*np.pi #Post processing: all thetas in [0,2*pi)

    #for theta, i in enumerate(thetas):
    #    if jointLimits[i][1] < theta < jointLimits[i][0]:
    #        print("Joint number: ", i+1, "is not within the limits")
    #        return False
    #return True
    for i in range(0,len(thetas)):
        if jointLimits[i,1] < thetas[i] or thetas[i] < jointLimits[i,0]:
            print("Joint number: ", i+1, "is not within the limits")
            return False
    return True


def agilus_theta_23(T_sd):
    """
    Calculates theta 2 and 3 of the agilus 6R robot
    PARAMTERS:
    T_sd: The desired end effector pose
    RETURNS: floats, Thetas 2 and 3 for both elbow up and elbow down solutions.
    """

    Ps = ps_from_Tsd(T_sd)
    P2 = np.array([Ps[0],Ps[1],Ps[2]-400]) # The same as Ps, but now relative to joint 2. Needed to do the following trigonometrics

    # Define the edges of the constructed triangle:
    a = np.sqrt(420**2+35**2)
    c = 455
    b = np.sqrt((np.sqrt(P2[0]**2+P2[1]**2)-25)**2 + P2[2]**2)

    # Calculate the four angles needed:
    psi = np.arccos(420/a)
    phi = atan2(P2[2], sqrt(P2[0]**2 + P2[1]**2)-25)
    alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
    beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

    # Calculate the elbow up and elbow down solutions of theta2 and theta3
    theta2_down =  -(phi - alpha)
    theta3_down =  -(np.pi - beta - psi)
    theta2_up = -(alpha + phi)
    theta3_up = np.pi - (beta - psi)

    return float(N(theta2_up)), float(N(theta3_up)), float(N(theta2_down)), float(N(theta3_down))



def euler_nx_y_nx(R):
    """
    Calculates the Euler angles for rotations about (-x)y(-x)
    PARAMETERS:
    R: The desired rotation
    RETURNS:
    float, Three angles
    """
    theta_x1 = -atan2(R[1,0], -R[2,0])
    theta_y = atan2(sqrt(1-R[0,0]**2), R[0,0])
    theta_x2 = -atan2(R[0,1], R[0,2])

    return float(N(theta_x1)), float(N(theta_y)), float(N(theta_x2))


def agilus_analytical_IK(T_sd):
    """
    Computes the analytical inverse kinematics of the Agilus 6R robot.
    PARAMETERS:
    Tsd: The desired end-effector pose
    RETURNS: float arrays of joint values, elbow up and elbow down.
    """
    thetas_up = [0,0,0,0,0,0]
    thetas_down = [0,0,0,0,0,0]
    Ps = ps_from_Tsd(T_sd)

    # Theta 1
    thetas_up[0] = float(N(-atan2(Ps[1],Ps[0])))  # Minus sign since the axis of rotation is defined as -z.
    thetas_down[0] = thetas_up[0]

    # Thetas 2,3
    thetas_up[1], thetas_up[2], thetas_down[1], thetas_down[2] = agilus_theta_23(T_sd)

    # Thetas 4,5,6
     # Elbow down:
    T1 = exp6(Slist[:,0], -thetas_down[0])
    T2 = exp6(Slist[:,1], -thetas_down[1])
    T3 = exp6(Slist[:,2], -thetas_down[2])
    R_down = (T3@T2@T1@T_sd@np.linalg.inv(M))  # The remaining rotation needed, defined in s
    thetas_down[3], thetas_down[4], thetas_down[5] = euler_nx_y_nx(R_down)

     # Elbow up:
    T1 = exp6(Slist[:,0], -thetas_up[0])
    T2 = exp6(Slist[:,1], -thetas_up[1])
    T3 = exp6(Slist[:,2], -thetas_up[2])
    R_up = (T3@T2@T1@T_sd@np.linalg.inv(M))
    thetas_up[3], thetas_up[4], thetas_up[5] = euler_nx_y_nx(R_up)

    return thetas_up, thetas_down




In [94]:
M1=sp.Matrix([[0, 1, 0, 0],
             [1, 0, 0, 0],
             [0, 0, -1, 200],
             [0, 0, 0, 1]])

M2=sp.Matrix([[1, 0, 0, 25],
             [0, 0, 1, 0],
             [0, -1, 0, 400],
             [0, 0, 0, 1]])

M3=sp.Matrix([[1, 0, 0, 455+25],
             [0, 0, 1, 0],
             [0, -1, 0, 400],
             [0, 0, 0, 1]])
M4=sp.Matrix([[0, 0, -1, 455+25+200], #420
             [0, 1, 0, 0],
             [1, 0, 0, 400+35],
             [0, 0, 0, 1]])
M5=sp.Matrix([[1, 0, 0, 455+25+420],
             [0, 0, 1, 0],
             [0, -1, 0, 400+35],
             [0, 0, 0, 1]])
M6=sp.Matrix([[0, 0, -1, 455+25+420+80],
             [0, 1, 0, 0],
             [1, 0, 0, 400+35],
             [0, 0, 0, 1]])
Mlist = np.array([M1,M2,M3,M4,M5,M6], dtype=float)

om = sp.zeros(3,6)
om1 = om[:, 0] = M1[:3, 2]
om2 = om[:, 1] = M2[:3, 2]
om3 = om[:, 2] = M3[:3, 2]
om4 = om[:, 3] = M4[:3, 2]
om5 = om[:, 4] = M5[:3, 2]
om6 = om[:, 5] = M6[:3, 2]
q = sp.zeros(3,6)
q1 = q[:,0] = M1[:3, 3]
q2 = q[:,1] = M2[:3, 3]
q3 = q[:,2] = M3[:3, 3]
q4 = q[:,3] = M4[:3, 3]
q5 = q[:,4] = M5[:3, 3]
q6 = q[:,5] = M6[:3, 3]

Slist = np.array(Slist_maker(om,q),dtype=float)

M = np.array(M6,dtype=float)

jointLimits = np.array([[-180, 180], [-190, 45], [-120, 156], [-180, 180], [-90, 90], [-180, 180]]) #Assuming joint 5 has limits [-90, 90]

print(Slist)

[[   0.    0.    0.   -1.    0.   -1.]
 [   0.    1.    1.    0.    1.    0.]
 [  -1.    0.    0.    0.    0.    0.]
 [   0. -400. -400.    0. -435.    0.]
 [   0.    0.    0. -435.    0. -435.]
 [   0.   25.  480.    0.  900.    0.]]


OPPGAVE 3.2

In [172]:

#Thetas_gen can be modified to get a valid desired position T_sd
thetas_gen = np.array([1,-1,-1.3,0,2,3])
T_sd = mr.FKinSpace(M,Slist,thetas_gen)
print("T_sd\n", T_sd)

T_sd
 [[   0.27682   -0.81052   -0.51617   22.33008]
 [  -0.16994   -0.56999    0.80389  -34.77703]
 [  -0.94578   -0.13482   -0.29552 1096.38744]
 [   0.         0.         0.         1.     ]]


In [173]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

# Finding the thetas
thetas_up, thetas_down = agilus_analytical_IK(T_sd)

# Resulting end effector pose and Ps for elbow up:
T_up = mr.FKinSpace(M,Slist,thetas_up)
P_up = ps_from_Tsd(T_up)

# And for elbow down:
T_down = mr.FKinSpace(M,Slist,thetas_down)
P_down = ps_from_Tsd(T_down)

print(T_sd,'\n\n', T_up, '\n\n',T_down)

[[   0.27682   -0.81052   -0.51617   22.33008]
 [  -0.16994   -0.56999    0.80389  -34.77703]
 [  -0.94578   -0.13482   -0.29552 1096.38744]
 [   0.         0.         0.         1.     ]] 

 [[   0.27682   -0.81052   -0.51617   22.33008]
 [  -0.16994   -0.56999    0.80389  -34.77703]
 [  -0.94578   -0.13482   -0.29552 1096.38744]
 [   0.         0.         0.         1.     ]] 

 [[   0.27682   -0.81052   -0.51617   22.33008]
 [  -0.16994   -0.56999    0.80389  -34.77703]
 [  -0.94578   -0.13482   -0.29552 1096.38744]
 [   0.         0.         0.         1.     ]]


In [174]:
#Numerical solution
thetas_num, asd = mr.IKinSpace(Slist,M,T_sd,[-2,0,3,0,0,0],0.0001,0.0001)

thetas_num_p = [0,0,0,0,0,0] #the post processed solution
#Post process angles to be [-pi, pi]
for i in range(0,6):
    thetas_num_p[i] = thetas_num[i]%(2*np.pi)
    if thetas_num_p[i]>np.pi:
        thetas_num_p[i] = thetas_num_p[i]-2*np.pi
print(thetas_num_p)

[-2.1415926535897967, -0.8915425265220005, -1.3093068861368273, -3.1415926535896745, 0.6407432409309539, 2.9999999999998828]


In [175]:

print('Generating thetas:', thetas_gen, '\nGenerated T_sd: \n', T_sd, '\n\n Elbow up: ', np.round(thetas_up,3), '\n Elbow down: ', np.round(thetas_down,3), '\n Numerical: ', np.round(thetas_num_p,3), asd)

print("\nElbow down: ")
print("Elbow down solution viable: ", apply_joint_lim(jointLimits, thetas_down), "\n")

print("\nElbow up: ")
print("Elbow down solution viable: ", apply_joint_lim(jointLimits, thetas_up), "\n")


Generating thetas: [ 1.  -1.  -1.3  0.   2.   3. ] 
Generated T_sd: 
 [[   0.27682   -0.81052   -0.51617   22.33008]
 [  -0.16994   -0.56999    0.80389  -34.77703]
 [  -0.94578   -0.13482   -0.29552 1096.38744]
 [   0.         0.         0.         1.     ]] 

 Elbow up:  [-2.142 -2.22   1.476 -3.142  2.097  3.   ] 
 Elbow down:  [-2.142 -0.892 -1.309 -3.142  0.641  3.   ] 
 Numerical:  [-2.142 -0.892 -1.309 -3.142  0.641  3.   ] True

Elbow down: 
Elbow down solution viable:  True 


Elbow up: 
Joint number:  5 is not within the limits
Elbow down solution viable:  False 

